In [78]:
#Generates a .csv file containing ONLY city names using data from https://simplemaps.com/data/gb-cities
#Note - this code is specific to this the format used by this website.  The website contians a world database, where data of the same format can be retrieved on other countries.
import csv
file = open("gb.csv")
reader = csv.reader(file)
lines = len(list(reader))
file.close()
file = open("List of cities.csv", "w")
file.close()
for row in range(1,lines-1):
    file = open("gb.csv")
    reader = csv.reader(file)
    cityName = list(reader)[row][0]
    file.close()
    file = open("List of cities.csv", "a")
    file.write(cityName + ",")
    file.close()

file = open("gb.csv")
reader = csv.reader(file)
cityName = list(reader)[lines-1][0]
file.close()
file = open("List of cities.csv", "a")
file.write(cityName)
file.close()

In [129]:
#main
import csv
file = open("List of cities.csv", "r")
reader = csv.reader(file)
l = list(reader) #creates an array of cities
l[0] = removeDuplicates(l[0])
file.close()

file = open("failedQueries.txt", "w") #deletes file if already exists
file.close()

deleteDB() #deletes contents of db table if already exists

import requests
import os.path

for i in range(0,len(l[0])): #len(l[0])
    geoJ = getGeoJ(i)
    tempGeoJ = geoJ #stores the geoJ string in a temporary variable which is changed throughout the loop
    fail = True
    while True:
        start = tempGeoJ.find("{'type': 'Polygon', 'coordinates': [[") #searches for substring to identify end of metadata and start of GeoJSON
        if start == -1 or (tempGeoJ.find("{'type': 'MultiPolygon', 'coordinates': [[") < start and tempGeoJ.find("{'type': 'MultiPolygon', 'coordinates': [[") != -1):
            start = tempGeoJ.find("{'type': 'MultiPolygon', 'coordinates': [[") #if a MultiPolygon is found before a Polygon, then this substring is searched for instead.
        end = tempGeoJ.find("]]}") + 3 #searches for substring to identify end of GeoJSON
        if start == -1: 
            if fail == True: #if the substring is not found on the first attempt, then the query does not return any Polygons or MultiPolygons, and is logged as a failure.
                failLog()
            break
                
        displayName = getDisplayName(tempGeoJ)
        region = getRegion(displayName)
        cityName = getCityName(displayName)
        parsedGeoJ = tempGeoJ[start:end]
        if os.path.isfile("C:/Users/Toan/Documents/CityGeoJSON/United Kingdom/GeoJSON files/" + displayName + ".json") == False:
            file = open("C:/Users/Toan/Documents/CityGeoJSON/United Kingdom/GeoJSON files/" + displayName + ".json", "w") 
            file.write(parsedGeoJ)
            file.close()
        else:
            duplicateFile()
        insertDB(cityName, region, displayName)
        tempGeoJ = tempGeoJ[end:]
        fail = False #If the loop has executed at least once without a failure, it does not need to be logged as a failure.

In [65]:
def getDisplayName(tempGeoJ): #This finds the full name of the city, by searching for the substring "display_name"
    dNameIndexes = []
    index = 0
    while index != -1:
        index = tempGeoJ.find("display_name", index+1)
        dNameIndexes.append(index) #Creates a list of indexes of all instances of "display_name" within tempGeoJ
    dNameIndexes.remove(-1)
    dNameIndexes.append(len(tempGeoJ))
    displayName = ""
     
    indexN = 0
    currentIndex = dNameIndexes[0]
    nextIndex = dNameIndexes[1]
    foundPolygon = False
    
    while currentIndex != (len(tempGeoJ)) and foundPolygon == False:
        if ("Polygon" in (tempGeoJ[currentIndex:nextIndex])) or ("MultiPolygon" in (tempGeoJ[currentIndex:nextIndex])):
            displayName = ""
            nameStart = currentIndex + 16
            char = tempGeoJ[nameStart] 
            if tempGeoJ[currentIndex + 15] == "'":
                while char != "'":
                    if char != "/":                
                        displayName = displayName + char
                    nameStart += 1
                    char = tempGeoJ[nameStart] 
                foundPolygon = True
            elif tempGeoJ[currentIndex + 15] == '"': #This is required since some display names have a ' within the name, so the string gets cut short.
                while char != '"':
                    if char != "/":                
                        displayName = displayName + char
                    nameStart += 1
                    char = tempGeoJ[nameStart] 
                foundPolygon = True
        else:
            indexN += 1
            currentIndex = dNameIndexes[indexN]
            try:
                nextIndex = dNameIndexes[indexN+1]
            except IndexError:
                break
    return displayName

In [66]:
def getRegion(displayName): #fetches the part of the display name between the first two commas
    region = ""
    for i in range(len(displayName)):
        if displayName[i] == ",":
            x = 1
            while displayName[i+x] != ",":
                region = region + displayName[i+x]
                x = x+1
            break
    return region

In [67]:
def getCityName(displayName): #fetches the part of the display name up until the first comma
    cityName = ""
    i = 0
    while displayName[i] != ",":
        cityName = cityName + displayName[i]
        i = i+1
    return cityName

In [68]:
def getGeoJ(i):
    searchCity = l[0][i]  #As it is from a .csv file, the array is 2D, i.e. [[London,Birmingham...,Broughton]] - hence 2 indexes are required
    api_url = "https://nominatim.openstreetmap.org/search.php?city=" + searchCity + "&country=United+Kingdom&polygon_geojson=1&format=json"
    response = requests.get(api_url) #retrieves geoJSON file from Nominatim API
    return str(response.json())


In [69]:
def failLog():
    file = open("failedQueries.txt", "a")
    file.write(l[0][i])
    file.write(geoJ)
    file.write("\n")
    file.write("Error: Request did not return Polygon")
    file.write("\n")
    file.write("\n")
    file.close()

In [70]:
def duplicateLog(cityName, region):
    file = open("failedQueries.txt", "a")
    file.write(cityName + ", " + region)
    file.write(parsedGeoJ)
    file.write("\n")
    file.write("Error: Duplicate [CountryIsoCode],[Region], [CityName]")
    file.write("\n")
    file.write("\n")
    file.close()


In [71]:
def insertDB(cityName, region, fileName):
    
    import pyodbc

    filePath = "'C:\\Users\\Toan\\Documents\\CityGeoJSON\\United Kingdom\\GeoJSON files\\" + fileName.replace("'","''") + ".json'"
    
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=HUNG-LAPTOP\TOAN;'
                          'Database=CityGeoJSON;'
                          'Trusted_Connection=yes;')

    cursor = conn.cursor()

    try:
        cursor.execute('''
                    INSERT INTO CityGeoJSON.dbo.CityGeoJSON (CountryName, CountryIsoCode, Region, CityName, GeoJson)
                    SELECT 'United Kingdom', 'GBR', ?, ?, * FROM OPENROWSET(BULK ''' + filePath + ''',SINGLE_CLOB) AS DATA;
                    ''',
                   (region.replace("'", "''"), cityName.replace("'","''"))
                  )
    except pyodbc.IntegrityError:
        duplicateLog(cityName, region)
        
    conn.commit()

In [72]:
def deleteDB():
    import pyodbc
    
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=HUNG-LAPTOP\TOAN;'
                          'Database=CityGeoJSON;'
                          'Trusted_Connection=yes;')
    

    cursor = conn.cursor()
    
    cursor.execute('''
                    DELETE CityGeoJSON.dbo.CityGeoJSON
                    ''')
    conn.commit()

In [73]:
def removeDuplicates(list):
    newList = []
    for i in list:
        if i not in newList:
            newList.append(i)
    return newList

In [124]:
def duplicateFile():
    startSearch = tempGeoJ.find(displayName)
    endSearch = tempGeoJ.find("Polygon", startSearch) 
    if tempGeoJ.find("'class': 'boundary', 'type': 'administrative'", startSearch, endSearch) != -1: #If second file has priority:
        file = open("C:/Users/Toan/Documents/CityGeoJSON/United Kingdom/GeoJSON files/" + displayName + ".json", "w") 
        file.write(parsedGeoJ) #overwrites first file
        file.close()
        
        
        import pyodbc
        conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=HUNG-LAPTOP\TOAN;'
                          'Database=CityGeoJSON;'
                          'Trusted_Connection=yes;') #delete first file from database
        cursor = conn.cursor()
        
        cursor.execute('''
                        SELECT GeoJSON FROM CityGeoJSON.dbo.CityGeoJSON WHERE CityName = ? AND Region = ? AND CountryIsoCode = 'GBR'
                        ''',
                       (cityName.replace("'","''"),region.replace("'","''"))
                      )
        failedGeoJ = cursor.fetchall() #Fetch the previous GeoJSON from the database in order to log it as a fail
        conn.commit()
        
        file = open("failedQueries.txt", "a") #Logging the previous file as a fail
        file.write(cityName + ", " + region) 
        file.write(failedGeoJ[0][0]) 
        file.write("\n")
        file.write("Error: Duplicate [CountryIsoCode],[Region], [CityName]")
        file.write("\n")
        file.write("\n")
        file.close()
        

        cursor.execute('''
                        DELETE FROM CityGeoJSON.dbo.CityGeoJSON WHERE CityName = ? AND Region = ? AND CountryIsoCode = 'GBR'
                        ''',
                       (cityName.replace("'","''"),region.replace("'","''"))
                      ) #Deletes the previous record so that the new file can be added without being logged as a fail.
        conn.commit()
        
